### Quick test script to compare the performance of the processing in jystilts to the one in these notebooks

Because this is just a comparison script, it might look a little dirtier. Sorry!

In [2]:
from astropy.table import Table, join
import warnings
from astropy.units import UnitsWarning
from function_package.load_and_clean_tables import _sanitise_table
import numpy as np
import os
import function_package as fp
import logging
logging.getLogger().setLevel("INFO")

logging.info("Modules loaded successfully")

INFO:root:Modules loaded successfully


In [3]:
# Initialise a region to test in:
ra_min, ra_max, dec_min, dec_max = 125, 130, -5, 0
REGION = fp.Region(ra_min, ra_max, dec_min, dec_max)
logging.info(REGION)

INFO:root:Region constrained to 125.0000 <= RA <= 130.0000 and -5.0000 <= DEC <= 0.0000.
This corresponds to a linear (!) size of 25 deg^2


In [5]:
# To perform the testing, please deposit a raw-matched file from the original code and a processed file from the
# original code in data/match_backups/tests_for_comparison/
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UnitsWarning)
    MATCH_TABLE = Table.read(f"{fp.DATAPATH}match_backups/tests_for_comparison/baseline_input_raw_match.fits")
MATCH_TABLE = _sanitise_table(MATCH_TABLE, REGION, "matched")
MATCH_TABLE.rename_column("type", "sweep_type")
MATCH_TABLE.rename_column("sweep_ident", "sweep_id")
MATCH_TABLE.rename_columns(["fflux", "nflux", "e_fflux", "e_nflux"], ["flux_fuv", "flux_nuv", "flux_err_fuv", "flux_err_nuv"])
MATCH_TABLE.rename_column("ebv_galex", "galex_ebv")
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UnitsWarning)
    POINTLIKE_COMP = Table.read(f"{fp.DATAPATH}match_backups/tests_for_comparison/baseline_input_pointlike_processed.fits")
POINTLIKE_COMP = _sanitise_table(POINTLIKE_COMP, REGION, "comparison")

INFO:root:The reduced matched table provided contains 975 sources.
INFO:root:The reduced comparison table provided contains 903 sources.


In [6]:
# Test the performance of the new sweep processing:
SWEEP_CORR = fp.process_sweep_columns(MATCH_TABLE)
plike, ext = fp.split_table_by_sourcetype(SWEEP_CORR)
band = "g"
from function_package.custom_constants import ALL_SWEEP_BANDS
for band in ALL_SWEEP_BANDS:
    print(band)
    print(np.max((plike[f"c_flux_{band}"] - POINTLIKE_COMP[f"c_flux_{band}"])/ plike[f"c_flux_{band}"]))
logging.info("As we can see, the relative errors are neglectible.")

INFO:root:As we can see, the relative errors are neglectible.


g
1.1616040043749564e-07
r
1.0551129522971926e-07
z
1.0810402448477127e-07
w1
1.0754281431780065e-07
w2
1.0678235412312255e-07
w3
1.0759226515561465e-07
w4
1.0884313731172959e-07


In [7]:
# Test the performance of the new Galex processing
try:
    MATCH_TABLE.rename_columns(["fflux", "nflux", "e_fflux", "e_nflux"], ["flux_fuv", "flux_nuv", "flux_err_fuv", "flux_err_nuv"])
    MATCH_TABLE.rename_column("ebv_galex", "galex_ebv")
except KeyError:
    pass
GALEX_CORR = fp.process_galex_columns(MATCH_TABLE)
plike, ext = fp.split_table_by_sourcetype(GALEX_CORR)
from function_package.custom_constants import ALL_GALEX_BANDS
for band in ALL_GALEX_BANDS:
    mask = POINTLIKE_COMP[f"c_flux_{band}"] > 0
    print(band)
    print(np.max((plike[mask][f"c_flux_{band}"] - POINTLIKE_COMP[mask][f"c_flux_{band}"])/ plike[mask][f"c_flux_{band}"]))
logging.info("As we can see, the relative errors are neglectible, but we need to watch out for NAN values")

INFO:root:As we can see, the relative errors are neglectible, but we need to watch out for NAN values


fuv
1.4809559830549247e-07
nuv
1.1365383890635863e-07


In [8]:
plike, ext = fp.split_table_by_sourcetype(MATCH_TABLE)
VHS_PLIKE = fp.process_vhs_columns(plike)
from function_package.custom_constants import ALL_VHS_BANDS
for band in ALL_VHS_BANDS:
    mask = POINTLIKE_COMP[f"c_flux_{band}"] > 0
    print(band)
    if np.sum(mask) > 0:
        print(np.max((VHS_PLIKE[mask][f"c_flux_{band}"] - POINTLIKE_COMP[mask][f"c_flux_{band}"])/ VHS_PLIKE[mask][f"c_flux_{band}"]))
logging.info("As we can see, the relative errors are neglectible, but we need to watch out for NAN values")

c:\Users\fabia\anaconda3\envs\4HIQ\lib\site-packages\numpy\ma\core.py:6900: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
INFO:root:As we can see, the relative errors are neglectible, but we need to watch out for NAN values


y
j
7.4732438287794575e-06
h
ks
7.59225810582596e-06


In [11]:
from astroquery.vsa import Vsa

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table, hstack
from astroquery.xmatch import XMatch
import os
import function_package as fp
import logging

In [5]:
Vsa.list_catalogs()
Vsa.all_databases

('VHSDR4',
 'VHSDR3',
 'VHSDR2',
 'VHSDR1',
 'VVVDR4',
 'VVVDR2',
 'VVVDR1',
 'VMCDR4',
 'VMCDR3',
 'VMCDR2',
 'VMCDR1',
 'VIKINGDR4',
 'VIKINGDR3',
 'VIKINGDR2',
 'VIDEODR5',
 'VIDEODR4',
 'VIDEODR3',
 'VIDEODR2',
 'VISTAOPENTIME')

In [ ]:
import astropy.coordinates as coord
import astropy.units as u
table = Vsa.query_region(coord.SkyCoord(10.625, -0.38,
                                           unit=(u.deg, u.deg),
                                           frame='galactic'),
                            radius=6 * u.arcsec)

In [15]:

ra_min, ra_max, dec_min, dec_max = 125, 130, -5, 0
REGION = fp.Region(ra_min, ra_max, dec_min, dec_max)
table1 = fp.load_and_clean_opt_agn_shu(REGION)

galex_table_id = "II/335/galex_ais"
matched_table = XMatch.query(cat1=table1,
                        cat2=f"vizier:{galex_table_id}",
                        max_distance=3.5 * u.arcsec, colRA1='ra',
                        colDec1='dec')

c:\Users\fabia\anaconda3\envs\4HIQ\lib\site-packages\astropy\io\ascii\core.py:1098: AstropyWarning: OverflowError converting to IntType in column objid, reverting to String.
  warnings.warn(
c:\Users\fabia\anaconda3\envs\4HIQ\lib\site-packages\astropy\io\ascii\core.py:1098: AstropyWarning: OverflowError converting to IntType in column phID, reverting to String.
  warnings.warn(
c:\Users\fabia\anaconda3\envs\4HIQ\lib\site-packages\astropy\io\ascii\core.py:1098: AstropyWarning: OverflowError converting to IntType in column primid, reverting to String.
  warnings.warn(
c:\Users\fabia\anaconda3\envs\4HIQ\lib\site-packages\astropy\io\ascii\core.py:1098: AstropyWarning: OverflowError converting to IntType in column primidd, reverting to String.
  warnings.warn(


In [21]:
matched_table

angDist,ra,dec,shu_z_phot,shu_prob_rf,RAJ2000,DEJ2000,name,objid,phID,Cat,RAfdeg,DEfdeg,FUVexp,NUVexp,GLON,GLAT,tile,img,sv,r.fov,Obs,b,E(B-V),Sp?,chkf,FUVmag,e_FUVmag,NUVmag,e_NUVmag,FUV.a,e_FUV.a,NUV.a,e_NUV.a,FUV.4,e_FUV.4,NUV.4,e_NUV.4,FUV.6,e_FUV.6,NUV.6,e_NUV.6,Fafl,Nafl,Fexf,Nexf,Fflux,e_Fflux,Nflux,e_Nflux,FXpos,FYpos,NXpos,NYpos,Fima,Nima,Fr,Nr,nS/G,fS/G,nell,fell,nPA,e_nPA,fPA,e_fPA,Fnr,F3r,Nar,Narms,Nbrms,Far,Farms,Fbrms,w_NUV,w_FUV,Prob,Sep,Nerr,Ferr,Ierr,Nperr,Fperr,CV,G,N,primid,groupid,Gd,Nd,primidd,groupidd,grouptot,OName,Size
float64,float64,float64,float64,float64,float64,float64,str22,str19,str19,str3,float64,float64,float64,float64,float64,float64,int32,int32,int32,float64,int32,int32,float64,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,int32,int32,str19,str39,int32,int32,str19,str39,str39,str1,float64
0.82569,125.111143024312,-4.66569333137881,0.8026694541558546,1.0,125.111156,-4.665922,GALEX J082026.6-043957,6378128634446611800,6378128634443464704,AIS,125.556935,-4.443879,182.0,182.0,227.7767,17.3861,50205,2465,77,0.496758,1,3,0.0526,0,0,20.5318,0.2251,20.2446,0.1205,1.7118,0.2251,0.1646,0.1205,1.9384,0.2494,0.2581,0.1122,1.7079,0.2544,0.1083,0.1312,0,1,0,0,22.247,4.61154,28.9834,3.2171,2984.54,1394.4,2984.71,1393.79,10.104,7.234,0.004,0.003,0.9929,0.9815,0.1077,0.4655,63.9,50.67,-22.8,-22.78,5.65834999,2.40739989,3.5,0.000857,0.000764,3.5,0.000962,0.000514,144.3,119.0,0.381,1.27,0.6,1.07,0.91,0.21,0.19,C,1,2,6378128634446611456,6378128634446611800+6378128636594099299,1,2,6378128634446611456,6378128634446611800+6378128636594099299,6378128634446611800+6378128636594099299,N,0.0
1.575823,125.077849881746,-4.65655919741207,1.202094771746106,1.0,125.077667,-4.656161,GALEX J082018.6-043922,6378128636592002105,6378128636590948352,AIS,125.085952,-5.201694,177.0,177.0,227.7502,17.3622,50205,2529,79,0.545595,1,1,0.0513,0,0,--,--,21.4582,0.2553,--,--,1.3782,0.2553,--,--,1.4512,0.2315,--,--,1.5214,0.3953,0,1,0,0,--,--,9.47808,2.22853,--,--,1933.42,3240.59,--,9.888,--,0.004,0.9799,0.0,0.2943,--,68.35,72.84,--,--,9.88829994,5.49293995,3.5,0.000819,0.000578,--,--,--,143.4,174.5,--,--,1.0,--,--,0.43,--,C,0,1,6378128636592002048,6378128636592002105,0,1,6378128636592002048,6378128636592002105,6378128636592002105,N,0.0
1.65823,125.299110272183,-4.99188386119016,1.5501156235984177,1.0,125.299238,-4.991441,GALEX J082111.8-045929,6378128636592000713,6378128636590948352,AIS,125.085952,-5.201694,177.0,177.0,228.1702,17.3831,50205,2529,79,0.298894,1,1,0.0543,0,0,--,--,21.2366,0.2196,--,--,1.1566,0.2196,--,--,1.1515,0.1983,--,--,1.0452,0.281,0,0,0,0,--,--,11.6242,2.35043,--,--,1403.66,2435.79,--,5.308,--,0.002,0.9789,0.0,0.1532,--,-39.83,-39.07,--,--,9.05045986,2.44722009,3.5,0.000685,0.00058,--,--,--,133.3,164.2,--,--,0.74,--,--,0.3,--,C,0,1,6378128636592001024,6378128636592000713,0,1,6378128636592001024,6378128636592000713,6378128636592000713,N,0.0
0.93419,125.326209946487,-4.88417265450998,0.9030894281886772,1.0,125.326241,-4.88443,GALEX J082118.2-045303,6378128634446610814,6378128634443464704,AIS,125.556935,-4.443879,182.0,182.0,228.0878,17.4605,50205,2465,77,0.496943,1,3,0.0517,0,0,20.3838,0.2071,19.8786,0.0979,1.5638,0.2071,-0.2014,0.0979,1.5662,0.1984,-0.1937,0.0918,1.5685,0.2359,-0.2056,0.1109,0,1,0,2,25.4955,4.86255,40.6018,3.66137,2470.01,870.25,2470.03,869.59,4.401,5.111,0.002,0.002,0.9839,0.9836,0.121,0.2024,45.81,44.56,23.4,23.41,5.76163006,2.55542994,3.5,0.000813,0.000715,3.5,0.000742,0.000592,139.7,118.2,0.234,1.31,0.54,0.88,0.77,0.14,0.14,C,1,2,6378128634446610432,6378128634446610814+6378128636594098399